# Building with LangChain

In [16]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()  # Load API keys and other sensitive data from the .env file

True

In [17]:
api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# LLM Chain

In [18]:
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instance

In [19]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in various ways:\n\n1. Automated testing: Langsmith can be used to write automated test scripts to test the functionality of the software. This can help in identifying bugs and issues in the code.\n\n2. Performance testing: Langsmith can be used to write performance tests to evaluate the performance of the software under different conditions. This can help in identifying performance bottlenecks and optimizing the code.\n\n3. Integration testing: Langsmith can be used to write integration tests to test the interaction between different components of the software. This can help in ensuring that all components work together seamlessly.\n\n4. Regression testing: Langsmith can be used to write regression tests to ensure that new changes or updates to the software do not break existing functionality. This can help in maintaining the stability of the software.\n\nOverall, Langsmith can help in improving the quality of the software by automati

In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [21]:
chain = prompt | llm 

In [22]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Test Automation: Langsmith can be used to automate testing processes, making it easier to run tests repeatedly and consistently. This can help in regression testing and ensuring that new code changes do not introduce bugs or issues.\n\n2. Test Data Generation: Langsmith can generate test data for various scenarios, allowing testers to cover a wide range of test cases without having to manually create each test data set.\n\n3. Performance Testing: Langsmith can be used to simulate large numbers of users or transactions to test the performance of an application under different load conditions.\n\n4. API Testing: Langsmith can be used to test APIs by sending requests and validating responses automatically. This can help in ensuring that APIs work as expected and meet the required specifications.\n\n5. Security Testing: Langsmith can be used to simulate various security threats and vulnerabilities to test the securit

In [23]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [24]:
chain = prompt | llm | output_parser

In [25]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in a variety of ways. Here are some ways in which Langsmith can assist with testing:\n\n1. **Automated Testing**: Langsmith can be used to automate testing processes by generating test cases and executing them against the software application. This can help in quickly identifying bugs and issues in the codebase.\n\n2. **Test Data Generation**: Langsmith can be used to generate synthetic test data that can be used to test the application under various scenarios. This can help in ensuring comprehensive test coverage.\n\n3. **Load Testing**: Langsmith can be used to simulate high loads on the application to test its performance under stress. This can help in identifying performance bottlenecks and optimizing the application for better scalability.\n\n4. **API Testing**: Langsmith can be used to test APIs by generating requests and verifying responses. This can help in validating the functionality and reliability of APIs.\n\n5. **Security Testi

# Retrieval Chain

In [26]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [28]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [30]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [31]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [32]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing and evaluation capabilities to assist with testing LLM applications.


# Conversation Retrieval Chain

In [33]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [34]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [35]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [36]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

# Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

The retriever we just created. This will let it easily answer questions about LangSmith
A search tool. This will let it easily answer questions that require up to date information.
First, let's set up a tool for the retriever we just created:

In [37]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [38]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [39]:
tools = [retriever_tool, search]

In [40]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [41]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workfl

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by providing capabilities for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. \n\nSome ways LangSmith can help with testing include:\n- Tracing: LangSmith offers tracing capabilities to help you trace and debug your LLM applications.\n- Evaluation: LangSmith provides evaluation capabilities to assess the performance and effectiveness of your LLM models during testing.\n\nYou can learn more about how LangSmith can assist with testing by exploring its user guide, setup instructions, pricing model, and additional resources available on the LangSmith platform.'}

In [42]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.theweathernetwork.com/us/weather/california/san-francisco', 'content': 'Get the most current and reliable weather information for San Francisco, California, including storm alerts, reports and videos. See the temperature, feels like, wind, humidity, visibility and more for the next 7 days.'}, {'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your area\nWe recognize our responsibility to use data and tec

{'input': 'what is the weather in SF?',
 'output': "I found some sources where you can check the weather in San Francisco:\n1. [The Weather Network - San Francisco, California](https://www.theweathernetwork.com/us/weather/california/san-francisco): Get the most current and reliable weather information for San Francisco, including storm alerts, reports, and videos for the next 7 days.\n2. [Weather.com - 10 Day Weather Forecast for San Francisco, CA](https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US): Check the 10-day weather forecast for San Francisco, including today's weather and upcoming conditions.\n3. [AccuWeather - San Francisco, CA Weather Forecast](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629): Get the current weather, radar, and air quality information for San Francisco, including the forecast for today, tonight, and the next two days.\n4. [Weather Underground - San Francisco, CA Weather Forecast](https://www.wunderground.com/f

In [43]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': "LangSmith is a platform for building production-grade LLM applications. It allows you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, the open-source framework for building with LLMs. You can learn more about LangSmith's workflows, setup, pricing, self-hosting options, tracing capabilities, evaluation capabilities, and more by referring to their user guide and resources. If you are interested in testing your LLM applications, LangSmith can be a valuable tool for that purpose."}